##Instalando Biblioteca externa do Collab necessária.

In [1]:
!pip install scikeras

In [2]:
import pandas as pd
import keras
import tensorflow
import tensorflow_hub as hub
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from scikeras.wrappers import KerasClassifier

##Importando Dataset

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/data.csv')

##Normalizando Dataset

In [4]:
dataset['clean_Sentence'] = dataset['Sentence'].str.lower()
dataset.loc[:,'clean_Sentence'] = dataset.loc[:,'clean_Sentence'].str.replace('[^\w\s]', '')
dataset.loc[:,'clean_Sentence'] = dataset.loc[:,'clean_Sentence'].str.replace('[0123456789]', '')
nltk.download('stopwords')
stop = stopwords.words('english')
dataset['clean_Sentence'] = dataset['clean_Sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
dataset

<ipython-input-4-d0aba0285ff0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.loc[:,'clean_Sentence'] = dataset.loc[:,'clean_Sentence'].str.replace('[^\w\s]', '')
<ipython-input-4-d0aba0285ff0>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.loc[:,'clean_Sentence'] = dataset.loc[:,'clean_Sentence'].str.replace('[0123456789]', '')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Sentence,Sentiment,clean_Sentence
0,The GeoSolutions technology will leverage Bene...,positive,geosolutions technology leverage benefon gps s...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,esi lows bk real possibility
2,"For the last quarter of 2010 , Componenta 's n...",positive,last quarter componenta net sales doubled eurm...
3,According to the Finnish-Russian Chamber of Co...,neutral,according finnishrussian chamber commerce majo...
4,The Swedish buyout firm has sold its remaining...,neutral,swedish buyout firm sold remaining percent sta...
...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,rising costs forced packaging producer huhtama...
5838,Nordic Walking was first used as a summer trai...,neutral,nordic walking first used summer training meth...
5839,"According shipping company Viking Line , the E...",neutral,according shipping company viking line eu deci...
5840,"In the building and home improvement trade , s...",neutral,building home improvement trade sales decrease...


##Dummyficando a coluna "Sentiment"

In [5]:
Y = dataset.loc[:,"Sentiment"]
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = keras.utils.to_categorical(encoded_Y)

##Vetorizando as entradas

In [11]:
cv = CountVectorizer()
sentences = dataset.loc[:, "clean_Sentence"]
X = cv.fit(sentences)
X = cv.transform(sentences)
x = X.toarray()

##Construção do modelo

In [7]:
def baseline_model():
 model = tensorflow.keras.Sequential()
 model.add(tensorflow.keras.layers.Dense(8, input_dim=10702, activation='relu'))
 model.add(tensorflow.keras.layers.Dropout(0.4))
 model.add(tensorflow.keras.layers.Dense(8, activation='relu'))
 model.add(tensorflow.keras.layers.Dropout(0.4))
 model.add(tensorflow.keras.layers.Dense(3, activation='softmax'))
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [8]:
estimator = KerasClassifier(model=baseline_model, epochs=100, batch_size=32, verbose=0)

In [9]:
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, X=x, y=dummy_y, cv=kfold, error_score='raise')

In [ ]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 63.35% (1.88%)
